This notebook is a our first approach to scraping the PubMed website, which takes a lot of time, and is a bad method. It's here as an archive.

# Loading libraries

In [4]:
import pip

def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])

import_or_install('datasets')

from datasets import load_dataset

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Download metadata

In [5]:
class MetaData:
  
  def __init__(self):
    self.HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
  
  def download_metadata(self, url):
    webpage = requests.get(url, headers=self.HEADERS)
    soup = BeautifulSoup(webpage.content, "lxml")
    table = soup.find("table")
    col_names = [i.text for i in table.find('tr').find_all('th')]
    if len(col_names) == 0:
      col_names = [str(i) for i in range(len(table.find('tr').find_all('td')))]
    df = pd.DataFrame(columns=col_names)
    for row in table.find_all('tr'):    
        columns_data = row.find_all('td')
        dict_data = {}
        if(len(columns_data) > 0):
            for col_index in range(len(columns_data)):
              dict_data[col_names[col_index]] = columns_data[col_index].text.strip()
            df = df.append(dict_data, ignore_index=True)
    return df

  @staticmethod
  def root_post_process(root_df, col_names = ['Root Category','Category Name']):
    col_mapper = dict(zip(root_df.columns, col_names))
    root_df = root_df.rename(col_mapper, axis=1)
    root_df = root_df.set_index(col_names[0])
    root_df['Category Split'] = root_df['Category Name'].apply(lambda x: x.replace(', and',' and ')).apply(lambda x: re.split(', | and ', x)).apply(lambda x: list(map(lambda y: y.lower(), x)))
    # .apply(lambda x: list(map(lambda y: y.lower(), x)))
    root_df['Category Name'] = root_df['Category Name'].apply(lambda x: x.lower())
    return root_df.explode('Category Split')

URL_ROOT = "https://www.nlm.nih.gov/bsd/indexing/training/MSH_010.html"
URL_SUBHEADINGS = 'https://www.nlm.nih.gov/mesh/qualifiers_scopenotes.html'
m = MetaData()
root_df = m.download_metadata(URL_ROOT)
subheadings_df = m.download_metadata(URL_SUBHEADINGS)
root_df = MetaData.root_post_process(root_df)
root_df

,Category Name,Category Split
Root Category,,
A,anatomy,anatomy
B,organisms,organisms
C,diseases,diseases
D,chemicals and drugs,chemicals
D,chemicals and drugs,drugs
E,"analytical, diagnostic and therapeutic techniq...",analytical
E,"analytical, diagnostic and therapeutic techniq...",diagnostic
E,"analytical, diagnostic and therapeutic techniq...",therapeutic techniques
E,"analytical, diagnostic and therapeutic techniq...",equipment


In [6]:
root_df.to_csv('root_categories.csv')

In [7]:
subheadings_df.to_csv('sub_categories.csv')

# Scraping Idea

In [ ]:
import requests
from bs4 import BeautifulSoup
webpage = requests.get('https://meshb.nlm.nih.gov/record/ui?ui=D022301', headers=({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'}))
soup = BeautifulSoup(webpage.content, "lxml")
dl = soup.find("dl").find('a').text
name = soup.find("dl").find('dd').text
print(dl, name)

B04.280.580.650.600.325 Feline Panleukopenia Virus


# Scraping is a bad idea

If we try to scrape from the website it takes too much time and is not the best method. Later, we found out that the MeSH website provides an XML file that we can parse to extract the information.

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
def stress_scraper(start, stop):
  processed_pair = []
  for i in tqdm_notebook(range(start, stop)):
    webpage = requests.get(f'https://meshb.nlm.nih.gov/record/ui?ui=D0{i:05d}', headers=({'User-Agent':
              'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
              'Accept-Language': 'en-US, en;q=0.5'}))
    soup = BeautifulSoup(webpage.content, "lxml")
    try:
      dl = soup.find("dl").find('a').text
      name = soup.find("dl").find('dd').text
      processed_pair.append([dl, name])
    except:
      pass
  return processed_pair
scraped_data = stress_scraper(28000, 30000)
import pandas as pd
pd.DataFrame(scraped_data).to_csv('data_first_tenthousand_15.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2000 [00:00<?, ?it/s]

ConnectionError: ignored